In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf 

2025-03-09 17:34:02.924346: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-09 17:34:02.985309: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-09 17:34:04.034269: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
###Data Prep 
##1.curl -L -o ~/Workspace/dp/A.I./ML/DL/tf/Models/toxic-tweets-dataset.zip  https://www.kaggle.com/api/v1/datasets/download/ashwiniyer176/toxic-tweets-dataset
##2.unzip toxic-tweets-dataset.zip && ls *.csv

data_df = pd.read_csv("FinalBalancedDataset.csv")
data_df = data_df.drop(columns=["Unnamed: 0"])
print(data_df.head(1))


   Toxicity                                              tweet
0         0   @user when a father is dysfunctional and is s...


In [3]:
data_df["Toxicity"].value_counts()

Toxicity
0    32592
1    24153
Name: count, dtype: int64

In [4]:
for _ in range(10):
    shuffle_data = np.random.randint(len(data_df))
    random_sample = data_df.iloc[shuffle_data]
    #print(random_sample["Toxicity"], random_sample["tweet"])

In [5]:
#PRE-PROCESSING
#TextToVector 

In [6]:
from transformers import BertTokenizer
tag = data_df["Toxicity"]
tag = np.array(tag)
tweet = data_df["tweet"]
tweet = np.array(tweet)
sentences = tweet.tolist()
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenized_output = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
print(tag.shape)
print(tweet.shape)
#print(tokenized_output)

/home/lab/miniconda3/envs/cancer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(56745,)
(56745,)


In [7]:
# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((tokenized_output, tag))

# Apply transformations
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)

In [10]:
#pipeline
train = dataset.take(int(len(dataset)* .7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))